# Retroactive Data Analysis

## Imports

In [1]:
import numpy
import os
import pandas
import statsmodels.api as sm
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append('/Users/jim/repos/poincare/axiom')

/Users/jim/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from axiom.vis.plotting import ggplot2

In [3]:
from axiom.vis.plotting.ggplot2 import image, gg2

In [4]:
from mpow import load_data, plotting, regression
plotting.output_notebook(hide_banner=True)

In [38]:
k = 3
fs = 90
fs2 = 75
image(gg2.ggplot(dgi) + gg2.aes_string(x='PainScore', y='Intake', color='SexDepressionCategory') +
      gg2.geom_point(size=5) + gg2.stat_smooth(method='lm', size=2) + 
      gg2.theme(**{'legend.position':'bottom', 'text':gg2.element_text(size=fs2),
                   'plot.title':gg2.element_text(hjust=0.5, size=fs),
                   'axis.text':gg2.element_text(size=fs2),
#                    'label':gg2.element_text(size=18),
                  }) + 
      gg2.labs(x='Pain Score', y='Intake (oMeq)', text_size=fs) + 
      gg2.ggtitle('Opioid Intake vs. Pain Score by Sex and Depression Status'), 
      height=k*700, width=k*1200, image_type='png', path='/Users/jim/desktop/mpow-gdi-largeer.png')

In [34]:
dgi['SexDepressionCategory'] = (1*dgi['FemDep'] + 
                                     2*dgi['MaleDep'] + 
                                     3*dgi['FemNon'] + 
                                     4*dgi['MaleNon']).apply({1:'Female Depressed    ',
                                                              2:'Male Depressed    ',
                                                              3:'Female Non-Depressed   ',
                                                              4:'Male Non-Depressed   '}.get)

## Load data

In [6]:
data = load_data.norm_daily_data()
intraday = load_data.norm_intraday_data()
details = load_data.norm_detail_data()

In [7]:
data.head()

,Patient,DayNum,Intake,PainScore,NumObs,AgeAtAdmit,Gender,ImpairmentGroup,Depression
0,1,1,80.0,29.0,7,32,Female,Spinal_Cord_Dysfunction,0
1,1,2,60.0,24.0,9,32,Female,Spinal_Cord_Dysfunction,0
2,1,3,70.0,38.0,11,32,Female,Spinal_Cord_Dysfunction,0
3,1,4,40.0,23.0,11,32,Female,Spinal_Cord_Dysfunction,0
4,2,1,75.0,63.0,14,56,Female,Spinal_Cord_Dysfunction,1


In [8]:
details.describe()

,Patient,AgeAtAdmit,Depression
count,154.000000,154.000000,154.000000
mean,77.500000,64.590909,0.344156
std,44.600075,15.506108,0.476642
min,1.000000,18.000000,0.000000
25%,39.250000,58.000000,0.000000
50%,77.500000,68.000000,0.000000
75%,115.750000,75.000000,1.000000
max,154.000000,99.000000,1.000000


In [9]:
details.groupby(['Gender', 'Depression'])['Patient'].count().to_frame('Count').reset_index()

,Gender,Depression,Count
0,Female,0,46
1,Female,1,44
2,Male,0,55
3,Male,1,9


In [10]:
details.Depression.value_counts().to_frame('Count').reset_index()

,index,Count
0,0,101
1,1,53


In [11]:
details.head()

,Patient,AgeAtAdmit,Gender,Depression,ImpairmentGroup
0,1,32,Female,0,Spinal_Cord_Dysfunction
1,2,56,Female,1,Spinal_Cord_Dysfunction
2,3,82,Male,0,Stroke
3,4,92,Male,0,Debility
4,5,67,Female,1,Orthopaedic_Disorders


## Explore details

### Age

Overall Distribution

In [12]:
plotting.show(plotting.histogram(details.AgeAtAdmit, bins=15))

Conditional on gender

In [13]:
details[['Gender', 'AgeAtAdmit']].groupby('Gender').median()

,AgeAtAdmit
Gender,
Female,68
Male,67


Conditional on depression

In [14]:
pandas.concat([details[details.Gender=='Male'].AgeAtAdmit.describe().to_frame('MaleAgeStats'),
details[details.Gender=='Female'].AgeAtAdmit.describe().to_frame('FemaleAgeStats')], axis=1)

,MaleAgeStats,FemaleAgeStats
count,64.000000,90.000000
mean,64.421875,64.711111
std,16.538368,14.821973
min,18.000000,25.000000
25%,58.000000,57.250000
50%,67.000000,68.000000
75%,75.000000,74.750000
max,92.000000,99.000000


In [15]:
details[['Gender', 'AgeAtAdmit']].groupby('Depression').median()

KeyError: 'Depression'

Conditional on impairment

In [16]:
details[['ImpairmentGroup', 'AgeAtAdmit']].groupby('ImpairmentGroup').median().sort_values('AgeAtAdmit')

,AgeAtAdmit
ImpairmentGroup,
Congenital_Deformities,25.0
Amputations,58.0
Neurologic_conditions,58.5
Pulmonary_Disorders,59.0
Burns,60.0
Brain_Dysfunction,60.5
Neuromuscular_disorders,64.0
Stroke,65.0
Debility,67.0


### Impairment

In [17]:
details.ImpairmentGroup.value_counts()

Orthopaedic_Disorders      51
Spinal_Cord_Dysfunction    35
Stroke                     17
Debility                   13
Amputations                 9
Cardiac                     7
Pulmonary_Disorders         5
Neuromuscular_disorders     5
Neurologic_conditions       4
Burns                       3
Major_Multiple_Trauma       2
Brain_Dysfunction           2
Congenital_Deformities      1
Name: ImpairmentGroup, dtype: int64

## Depression Gender Interaction

Model: $$\LARGE{s_{t,i} \sim \beta_{0} + \beta_{1}p_{t,i} + \alpha^{df} + \alpha^{nf} + \alpha^{dm} + \alpha^{nm} + \epsilon_{t,i}}$$

Sufficiency?

In [18]:
dgi_cats = {2:'FemNon',3:'MaleNon',4:'FemDep',6:'MaleDep'}
dgi_counts = (((details.Depression + 1) * ((details.Gender=='Male').astype(int) + 2))
            .to_frame('DepGenInter').applymap(dgi_cats.get))
dgi_counts.DepGenInter.value_counts()

MaleNon    55
FemNon     46
FemDep     44
MaleDep     9
Name: DepGenInter, dtype: int64

In [19]:
dgi = data.copy()
dgi['FemDep'] = ((dgi.Depression==1)&(dgi.Gender=='Female')).astype(int)
dgi['MaleDep'] = ((dgi.Depression==1)&(dgi.Gender=='Male')).astype(int)
dgi['FemNon'] = ((dgi.Depression==0)&(dgi.Gender=='Female')).astype(int)
dgi['MaleNon'] = ((dgi.Depression==0)&(dgi.Gender=='Male')).astype(int)

In [20]:
dgi['MPS'] = dgi['PainScore'] / dgi['NumObs']

In [21]:
dgi_model = regression.ols(dgi.dropna(), ['MPS', 'FemDep', 'MaleDep', 'FemNon', 'MaleNon'], 'Intake')
dgi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Intake   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     133.4
Date:                Sat, 28 Jul 2018   Prob (F-statistic):           5.45e-98
Time:                        10:51:00   Log-Likelihood:                -7376.4
No. Observations:                1551   AIC:                         1.476e+04
Df Residuals:                    1546   BIC:                         1.479e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.5244      1.099      6.849      0.000       5.369       9.680
MPS            7.8207      0.380     20.587      0.000       7.076       8.566
FemDep        12.2337      1.316      9.295      0.000       9.652      14.815
MaleDep       -1.6211      2.505     -0.647      0.518      -6.534       3.292
FemNon         0.1522      1.334      0.114      0.909      -2.464       2.768
MaleNon       -3.2404      1.158     -2.798      0.005      -5.512      -0.968
==============================================================================
Omnibus:                      795.556   Durbin-Watson:                   0.489
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6508.123
Skew:                           2.258   Prob(JB):                         0.00
Kurtosis:                      11.962   Cond. No.                     2.54e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.58e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [22]:
bse = dict(dgi_model.bse)
for param, coeff in dict(dgi_model.params).items():
    print('{}:\t{:.3f}\t({:.3f}, {:.3f})'.format(param[:5], coeff, coeff - 1.96*bse[param], coeff + 1.96*bse[param]))

const:	7.524	(5.371, 9.678)
MPS:	7.821	(7.076, 8.565)
FemDe:	12.234	(9.654, 14.813)
MaleD:	-1.621	(-6.530, 3.288)
FemNo:	0.152	(-2.462, 2.766)
MaleN:	-3.240	(-5.511, -0.970)


In [23]:
dgi.head()

,Patient,DayNum,Intake,PainScore,NumObs,AgeAtAdmit,Gender,ImpairmentGroup,Depression,FemDep,MaleDep,FemNon,MaleNon,MPS
0,1,1,80.0,29.0,7,32,Female,Spinal_Cord_Dysfunction,0,0,0,1,0,4.142857
1,1,2,60.0,24.0,9,32,Female,Spinal_Cord_Dysfunction,0,0,0,1,0,2.666667
2,1,3,70.0,38.0,11,32,Female,Spinal_Cord_Dysfunction,0,0,0,1,0,3.454545
3,1,4,40.0,23.0,11,32,Female,Spinal_Cord_Dysfunction,0,0,0,1,0,2.090909
4,2,1,75.0,63.0,14,56,Female,Spinal_Cord_Dysfunction,1,1,0,0,0,4.500000


In [24]:
beta0, beta1, femdep, maledep, femnon, malenon = dgi_model.params

In [25]:
beta0, beta1, femdep, maledep, femnon, malenon

(7.5244191685065518,
 7.820745971164059,
 12.233696453273195,
 -1.6210956075009102,
 0.15223085315825141,
 -3.2404125304239426)

In [26]:
def dgi_estimate(p, df, nf, dm, nm):
    return beta0 + beta1*p + femdep*df + femnon*nf + maledep*dm + malenon*nm

In [27]:
ps = numpy.arange(0, 10.1, 0.1)
fit_df = pandas.concat([pandas.DataFrame({'MPS':ps, 'Intake':dgi_estimate(ps, 0, 0, 1, 0), 'DGICategory': len(ps)*['MaleDep']}),
                        pandas.DataFrame({'MPS':ps, 'Intake':dgi_estimate(ps, 0, 0, 0, 1), 'DGICategory': len(ps)*['MaleNon']}),
                        pandas.DataFrame({'MPS':ps, 'Intake':dgi_estimate(ps, 1, 0, 0, 0), 'DGICategory': len(ps)*['FemDep']}),
                        pandas.DataFrame({'MPS':ps, 'Intake':dgi_estimate(ps, 0, 1, 0, 0), 'DGICategory': len(ps)*['FemNon']})], axis=0)

In [28]:
fit_df['DataType'] = 'Fitted'

In [29]:
fit_df.head()

,DGICategory,Intake,MPS,DataType
0,MaleDep,5.903324,0.0,Fitted
1,MaleDep,6.685398,0.1,Fitted
2,MaleDep,7.467473,0.2,Fitted
3,MaleDep,8.249547,0.3,Fitted
4,MaleDep,9.031622,0.4,Fitted


In [30]:
dgi[dgi.FemDep==1][['Intake', 'MeanPainScore']]

KeyError: "['MeanPainScore'] not in index"

In [31]:
fit_df

,DGICategory,Intake,MPS,DataType
0,MaleDep,5.903324,0.0,Fitted
1,MaleDep,6.685398,0.1,Fitted
2,MaleDep,7.467473,0.2,Fitted
3,MaleDep,8.249547,0.3,Fitted
4,MaleDep,9.031622,0.4,Fitted
5,MaleDep,9.813697,0.5,Fitted
6,MaleDep,10.595771,0.6,Fitted
7,MaleDep,11.377846,0.7,Fitted
8,MaleDep,12.159920,0.8,Fitted
9,MaleDep,12.941995,0.9,Fitted


### Expl. vis

In [32]:
tmp = dgi.copy()
tmp = tmp[['MeanPainScore', 'Intake', 'Gender', 'Depression']]
tmp['Gender'] = (tmp['Gender']=='Male').astype(int) + 1
tmp['Depression'] = tmp['Depression'] + 2
tmp['DepGenInteraction'] = (tmp['Depression'] * tmp['Gender']).apply({2:'FemNon',3:'FemDep',4:'MaleNon',6:'MaleDep'}.get)

KeyError: "['MeanPainScore'] not in index"

In [33]:
image(gg2.ggplot(tmp[['MeanPainScore', 'Intake', 'DepGenInteraction']]) + 
      gg2.aes_string(x='MeanPainScore', y='Intake', color='DepGenInteraction') + 
      gg2.geom_point() + gg2.stat_smooth(method='lm'))

KeyError: "['MeanPainScore' 'DepGenInteraction'] not in index"